# 第2章: グラフの制御フロー

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-2.0-flash", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = (
                 getattr(llm, 'model', None) or
                 getattr(llm, 'model_name', None) or
                 getattr(llm, 'model_id', None) or
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    )
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

### ■ 問題001: 条件付きエッジによる分岐の導入

LangGraphの強力な機能の一つである条件付きエッジを導入し、グラフの実行パスを動的に制御する方法を学びましょう。ここでは、入力された数値が偶数か奇数かによって、異なる処理を行うグラフを作成します。

*   **学習内容:** `add_conditional_edges`を使用して、グラフの実行パスを動的に制御する方法を学びます。ルーター関数がどのように次のノードを決定するのか、そして状態が分岐間でどのように共有されるかを理解します。

In [ ]:
# 解答欄001 - グラフ構築
____ typing ____ ____, ____
____ langgraph.graph ____ ____, ____
____ langchain_core.messages ____ ____ # AIMessageは解答例で使うのでここでは不要かも

# 状態定義
____ ConditionalState(____):
    number: int
    message: str

# ノード定義
____ check_number_node(state: ConditionalState):
    print(f"入力された数値: {state['number']}")
    ____ {} # 何も更新しない

____ even_node(state: ConditionalState):
    msg = f"{state['number']} は偶数です。"
    print(msg)
    ____ {"message": msg}

____ odd_node(state: ConditionalState):
    msg = f"{state['number']} は奇数です。"
    print(msg)
    ____ {"message": msg}

# ルーター関数
____ route_by_parity(state: ConditionalState):
    if state["number"] % 2 == 0:
        return "to_even" # 偶数ならこの名前を返す (解答例より)
    else:
        return "to_odd" # 奇数ならこの名前を返す (解答例より)

# グラフ構築
workflow = StateGraph(ConditionalState)
workflow.add_node("check", check_number_node)
workflow.add_node("even", even_node)
workflow.add_node("odd", odd_node)
workflow.set_entry_point("check")

# 条件付きエッジの追加
workflow.add_conditional_edges(
    "check",
    route_by_parity,
    {
        "to_even": "even",
        "to_odd": "odd" # 解答例より
    }
)

workflow.add_edge("even", END)
workflow.add_edge("odd", END)
graph = workflow.compile()

In [ ]:
# 解答欄001 - グラフ可視化
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# 解答欄001 - グラフ実行
print("--- 偶数のテスト ---")
print(f"最終結果: {graph.invoke({'number': 42, 'message': ''})}") # messageを初期化
print("--- 奇数のテスト ---")
print(f"最終結果: {graph.invoke({'number': 77, 'message': ''})}") # messageを初期化

<details><summary>解答001</summary>

``````python
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langchain_core.messages import AIMessage

# 状態定義
class ConditionalState(TypedDict):
    number: int
    message: str

# ノード定義
def check_number_node(state: ConditionalState):
    print(f"入力された数値: {state['number']}")
    return {} # 何も更新しない

def even_node(state: ConditionalState):
    msg = f"{state['number']} は偶数です。"
    print(msg)
    return {"message": msg}

def odd_node(state: ConditionalState):
    msg = f"{state['number']} は奇数です。"
    print(msg)
    return {"message": msg}

# ルーター関数
def route_by_parity(state: ConditionalState):
    if state["number"] % 2 == 0:
        return "to_even" # 偶数ならこの名前を返す
    else:
        return "to_odd" # 奇数ならこの名前を返す

# グラフ構築
workflow = StateGraph(ConditionalState)
workflow.add_node("check", check_number_node)
workflow.add_node("even", even_node)
workflow.add_node("odd", odd_node)
workflow.set_entry_point("check")

# 条件付きエッジの追加
workflow.add_conditional_edges(
    "check",
    route_by_parity,
    {
        "to_even": "even",
        "to_odd": "odd"
    }
)

workflow.add_edge("even", END)
workflow.add_edge("odd", END)
graph = workflow.compile()

# 実行
print("--- 偶数のテスト ---")
print(f"最終結果: {graph.invoke({'number': 42, 'message': ''})}")
print("--- 奇数のテスト ---")
print(f"最終結果: {graph.invoke({'number': 77, 'message': ''})}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント

*   **ルーター関数:** `route_by_parity` 関数は、現在のグラフの状態 `state` を受け取り、次に進むべきエッジの名前（文字列）を返します。この戻り値に基づいて、`add_conditional_edges` で定義したマッピングに従い、処理が分岐します。
*   **`add_conditional_edges`:** 3つの主要な引数を取ります。
    1.  **始点ノード (`"check"`)**: 分岐を開始するノード。
    2.  **ルーター関数 (`route_by_parity`)**: どのパスに進むかを決定する関数。
    3.  **パスのマッピング (辞書)**: ルーター関数の戻り値（キー）と、次に遷移するノード名（バリュー）の対応を定義します。

---</details>

### ■ 問題002: 状態を使ったループ（シンプルなカウンター制御）
*   **学習内容:** 状態と条件付きエッジを組み合わせて、カウンターが上限に達するまで処理を繰り返すループ構造を実装します。

In [ ]:
# 解答欄002 - グラフ構築
____ typing ____ ____, ____
____ langgraph.graph ____ ____, ____
____ langchain_core.messages ____ ____ # AIMessageは解答例で使用

# 状態定義
____ CounterLoopState(____):
    count: int
    max_count: int
    log: list[str]

# ノード定義
____ increment_node(state: CounterLoopState):
    new_count = state["count"] + 1
    log_message = f"カウンターが {new_count} になりました。"
    print(log_message)
    ____ {"count": new_count, "log": state["log"] + [log_message]}

____ final_message_node(state: CounterLoopState):
    final_msg = f"ループ終了。最終カウント: {state['count']}"
    print(final_msg)
    ____ {"log": state["log"] + [final_msg]}

# ルーター関数
____ should_continue(state: CounterLoopState):
    ____ state["count"] < state["max_count"]:
        ____ "continue_loop" # ループ継続のキー (解答例より)
    else:
        return "exit_loop" # ループ終了のキー (解答例より)

# グラフ構築
workflow = StateGraph(CounterLoopState)
workflow.add_node("increment", increment_node)
workflow.add_node("final_message", final_message_node)

workflow.set_entry_point("increment")

workflow.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue_loop": "increment",
        "exit_loop": "final_message" # 解答例より
    }
)
workflow.add_edge("final_message", END)
graph = workflow.compile()

In [ ]:
# 解答欄002 - グラフ可視化
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# 解答欄002 - グラフ実行
initial_state = {"count": 0, "max_count": 3, "log": []}
print(f"--- カウンター上限 {initial_state['max_count']} のループテスト --- ")
final_result = graph.invoke(initial_state)
print(f"最終状態のログ: {final_result['log']}")

<details><summary>解答002</summary>

``````python
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langchain_core.messages import AIMessage

# 状態定義
class CounterLoopState(TypedDict):
    count: int
    max_count: int
    log: list[str]

# ノード定義
def increment_node(state: CounterLoopState):
    new_count = state["count"] + 1
    log_message = f"カウンターが {new_count} になりました。"
    print(log_message)
    return {"count": new_count, "log": state["log"] + [log_message]}

def final_message_node(state: CounterLoopState):
    final_msg = f"ループ終了。最終カウント: {state['count']}"
    print(final_msg)
    return {"log": state["log"] + [final_msg]}

# ルーター関数
def should_continue(state: CounterLoopState):
    if state["count"] < state["max_count"]:
        return "continue_loop"
    else:
        return "exit_loop"

# グラフ構築
workflow = StateGraph(CounterLoopState)
workflow.add_node("increment", increment_node)
workflow.add_node("final_message", final_message_node)

workflow.set_entry_point("increment")

workflow.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue_loop": "increment",
        "exit_loop": "final_message"
    }
)
workflow.add_edge("final_message", END)
graph = workflow.compile()

# 実行
initial_state = {"count": 0, "max_count": 3, "log": []}
print(f"--- カウンター上限 {initial_state['max_count']} のループテスト --- ")
final_result = graph.invoke(initial_state)
print(f"最終状態のログ: {final_result['log']}")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント

*   **ループの実現:** `increment_node` の処理後、`should_continue` ルーターが呼び出されます。このルーターが `"continue_loop"` を返すと、処理は再び `increment_node` に戻ります。これがループバックエッジとなり、繰り返し処理を実現します。
*   **状態による制御:** ループを継続するかどうかは、状態 `CounterLoopState` の `count` と `max_count` の比較によって決定されます。状態がループの振る舞いを制御する中心的な役割を果たします。
*   **終了条件:** `count` が `max_count` に達すると、ルーターは `"exit_loop"` を返し、処理は `final_message_node` に移り、その後 `END` でグラフが終了します。

---</details>

### ■ 問題003: LLMによるループ継続判断（自己反省ループ）
*   **学習内容:** LLM自身が生成内容を評価し、特定の品質基準を満たすまで、あるいは最大試行回数に達するまで再生成を繰り返す、より高度なループを構築します。

In [ ]:
# 解答欄003 - グラフ構築
____ typing ____ ____, ____, Optional
____ langgraph.graph ____ ____, ____
____ langchain_core.messages ____ ____, ____

# 状態定義
____ ReflectionLoopState(____):
    topic: str
    current_summary: Optional[str]
    critique: Optional[str]
    attempts_left: int
    max_attempts: int
    log: list[str]

# ノード定義
____ generate_summary_node(state: ReflectionLoopState):
    prompt = f"以下のトピックについて、非常に短い（1文程度の）要約を作成してください: {state['topic']}"
    ____ state.get("critique") and "完璧" not in state["critique"].lower(): # 「完璧」でない批判がある場合のみ考慮 (解答例より)
        prompt += f"
前回の要約への批判: {state['critique']}
この批判を踏まえて改善してください。"
    
    print(f"
[要約生成ノード] プロンプト: {prompt}")
    response = llm.____(prompt) # llmは準備セルで初期化済み
    summary = response.content.strip()
    log_msg = f"生成された要約: '{summary}'"
    print(log_msg)
    ____ {"current_summary": summary, "log": state["log"] + [log_msg], "critique": None} # critiqueをリセット (解答例より)

____ critique_summary_node(state: ReflectionLoopState):
    summary = state["current_summary"]
    prompt = f"以下の要約を評価してください。改善点があれば具体的に指摘し、もし完璧だと思えば「完璧」とだけ答えてください。要約: '{summary}'"
    print(f"
[評価ノード] プロンプト: {prompt}")
    response = llm.____(prompt)
    critique_text = response.content.strip()
    log_msg = f"評価結果: '{critique_text}'"
    print(log_msg)
    new_attempts_left = state["attempts_left"] - 1
    return {"critique": critique_text, "log": state["log"] + [log_msg], "attempts_left": new_attempts_left}

def final_result_node(state: ReflectionLoopState):
    final_log = f"
[最終結果ノード] 最終的な要約: '{state['current_summary']}', 残り試行回数: {state['attempts_left']}"
    print(final_log)
    return {"log": state["log"] + [final_log]}

# ルーター関数
def should_reflect_or_finish(state: ReflectionLoopState):
    critique = state.get("critique", "")
    if "完璧" in critique.lower(): # 解答例より
        print("  -> 評価が「完璧」のため終了します。")
        return "finish"
    elif state["attempts_left"] <= 0:
        print("  -> 最大試行回数に達したため終了します。")
        return "finish"
    else:
        print("  -> 改善の余地あり、かつ試行回数が残っているため再生成します。")
        return "regenerate" # 再生成のキー (解答例より)

# グラフ構築
workflow = StateGraph(ReflectionLoopState)
workflow.add_node("generate_summary", generate_summary_node)
workflow.add_node("critique_summary", critique_summary_node)
workflow.add_node("final_result", final_result_node)

workflow.set_entry_point("generate_summary")
workflow.add_edge("generate_summary", "critique_summary")

workflow.add_conditional_edges(
    "critique_summary",
    should_reflect_or_finish,
    {
        "regenerate": "generate_summary", # 再生成の場合はこのノードへ (解答例より)
        "finish": "final_result"
    }
)
workflow.add_edge("final_result", END)
graph = workflow.compile()

In [ ]:
# 解答欄003 - グラフ可視化
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# 解答欄003 - グラフ実行
topic_to_summarize = "大規模言語モデルの最新の進展について"
max_reflection_attempts = 2
initial_reflection_state = {
    "topic": topic_to_summarize,
    "current_summary": None,
    "critique": None,
    "attempts_left": max_reflection_attempts,
    "max_attempts": max_reflection_attempts,
    "log": []
}
print(f"--- 自己反省ループテスト (トピック: {topic_to_summarize}, 最大試行回数: {max_reflection_attempts}) ---")
final_reflection_result = graph.invoke(initial_reflection_state, {"recursion_limit": 10}) # recursion_limitを適切に設定
print(f"
最終状態のログ:")
for log_entry in final_reflection_result['log']:
    print(f"  - {log_entry}")

<details><summary>解答003</summary>

``````python
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage # AIMessageは直接は使わないが、llm応答がAIMessageなのでimportは維持

# 状態定義
class ReflectionLoopState(TypedDict):
    topic: str
    current_summary: Optional[str]
    critique: Optional[str]
    attempts_left: int
    max_attempts: int
    log: list[str]

# ノード定義
def generate_summary_node(state: ReflectionLoopState):
    prompt = f"以下のトピックについて、非常に短い（1文程度の）要約を作成してください: {state['topic']}"
    if state.get("critique") and "完璧" not in state["critique"].lower(): # 「完璧」でない批判がある場合のみ考慮
        prompt += f"
前回の要約への批判: {state['critique']}
この批判を踏まえて改善してください。"
    
    print(f"
[要約生成ノード] プロンプト: {prompt}")
    response = llm.invoke(prompt)
    summary = response.content.strip()
    log_msg = f"生成された要約: '{summary}'"
    print(log_msg)
    return {"current_summary": summary, "log": state["log"] + [log_msg], "critique": None} 

def critique_summary_node(state: ReflectionLoopState):
    summary = state["current_summary"]
    prompt = f"以下の要約を評価してください。改善点があれば具体的に指摘し、もし完璧だと思えば「完璧」とだけ答えてください。要約: '{summary}'"
    print(f"
[評価ノード] プロンプト: {prompt}")
    response = llm.invoke(prompt)
    critique_text = response.content.strip()
    log_msg = f"評価結果: '{critique_text}'"
    print(log_msg)
    new_attempts_left = state["attempts_left"] - 1
    return {"critique": critique_text, "log": state["log"] + [log_msg], "attempts_left": new_attempts_left}

def final_result_node(state: ReflectionLoopState):
    final_log = f"
[最終結果ノード] 最終的な要約: '{state['current_summary']}', 残り試行回数: {state['attempts_left']}"
    print(final_log)
    return {"log": state["log"] + [final_log]}

# ルーター関数
def should_reflect_or_finish(state: ReflectionLoopState):
    critique = state.get("critique", "")
    if "完璧" in critique.lower():
        print("  -> 評価が「完璧」のため終了します。")
        return "finish"
    elif state["attempts_left"] <= 0:
        print("  -> 最大試行回数に達したため終了します。")
        return "finish"
    else:
        print("  -> 改善の余地あり、かつ試行回数が残っているため再生成します。")
        return "regenerate"

# グラフ構築
workflow = StateGraph(ReflectionLoopState)
workflow.add_node("generate_summary", generate_summary_node)
workflow.add_node("critique_summary", critique_summary_node)
workflow.add_node("final_result", final_result_node)

workflow.set_entry_point("generate_summary")
workflow.add_edge("generate_summary", "critique_summary")

workflow.add_conditional_edges(
    "critique_summary",
    should_reflect_or_finish,
    {
        "regenerate": "generate_summary",
        "finish": "final_result"
    }
)
workflow.add_edge("final_result", END)
graph = workflow.compile()

# 実行
topic_to_summarize = "LangGraphライブラリの主な機能と利点について"
max_reflection_attempts = 2 
initial_reflection_state = {
    "topic": topic_to_summarize,
    "current_summary": None,
    "critique": None,
    "attempts_left": max_reflection_attempts,
    "max_attempts": max_reflection_attempts,
    "log": []
}
print(f"--- 自己反省ループテスト (トピック: {topic_to_summarize}, 最大試行回数: {max_reflection_attempts}) ---")
final_reflection_result = graph.invoke(initial_reflection_state, {"recursion_limit": 15}) # ループがあるのでrecursion_limitを増やす
print(f"
最終状態のログ:")
for log_entry in final_reflection_result['log']:
    print(f"  - {log_entry}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント

*   **LLMによる評価と改善指示:** `critique_summary_node` がLLMを使って現在の要約を評価し、改善点を指摘します（または「完璧」と判断します）。`generate_summary_node` は、この批判（`critique`）を次のプロンプトに含めることで、より良い要約を生成しようと試みます。
*   **ループ制御:** `should_reflect_or_finish` ルーターが、評価結果（`critique`）と残り試行回数（`attempts_left`）に基づいてループを継続するか（`"regenerate"` で `generate_summary_node` に戻る）、終了するか（`"finish"` で `final_result_node` に進む）を決定します。
*   **状態の役割:** `current_summary`（最新の生成物）、`critique`（LLMからのフィードバック）、`attempts_left`（ループ終了条件）といった状態が、この自己反省ループの中心的な役割を担います。
*   **`recursion_limit`:** ループ構造を持つグラフを実行する場合、`invoke` メソッドの `config` 引数で `recursion_limit` を適切に設定することが重要です。デフォルト値ではループの深さが足りずにエラーになることがあります。

---</details>

### ■ 問題004: 並列処理（ファンアウト・ファンイン）による複数タスクの同時実行
*   **学習内容:** 1つの始点ノードから複数のノードへ処理を分岐（ファンアウト）させ、それらの完了を待ってから結果を統合（ファンイン）する方法を学びます。

In [ ]:
# 解答欄004 - グラフ構築
____ typing ____ ____, List, Optional
____ langgraph.graph ____ ____, ____

# 状態定義
____ FanOutFanInState(____):
    input_data: str
    processed_a: Optional[str]
    processed_b: Optional[str]
    final_output: Optional[str]

# ノード定義
____ start_node(state: FanOutFanInState):
    print(f"[開始ノード] 入力データ: {state['input_data']}")
    ____ {}

____ process_a_node(state: FanOutFanInState):
    # ここではダミー処理として入力に "_A" を追加
    result_a = state["input_data"] + "_ProcessedA"
    print(f"  [処理Aノード] 結果: {result_a}")
    ____ {"processed_a": result_a}

____ process_b_node(state: FanOutFanInState):
    # ここではダミー処理として入力に "_B" を追加
    result_b = state["input_data"] + "_ProcessedB"
    print(f"  [処理Bノード] 結果: {result_b}")
    ____ {"processed_b": result_b}

____ aggregate_node(state: FanOutFanInState):
    # process_a_node と process_b_node の両方の完了を待って実行される
    result = f"集約結果: A='{state.get('processed_a', 'N/A')}', B='{state.get('processed_b', 'N/A')}'"
    print(f"[集約ノード] {result}")
    ____ {"final_output": result}

# グラフ構築
workflow = ____(FanOutFanInState)
workflow.____("start", start_node)
workflow.____("process_a", process_a_node)
workflow.add_node("process_b", process_b_node)
workflow.add_node("aggregate", aggregate_node)

workflow.set_entry_point("start")

# ファンアウト: start から process_a と process_b へ
workflow.add_edge("start", "process_a")
workflow.add_edge("start", "process_b") # 解答例より

# ファンイン: process_a と process_b の両方が完了したら aggregate へ
workflow.add_edge(["process_a", "process_b"], "aggregate") # 解答例より

workflow.add_edge("aggregate", END)
graph = workflow.compile()

In [ ]:
# 解答欄004 - グラフ可視化
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# 解答欄004 - グラフ実行
initial_fan_state = {"input_data": "初期データ", "processed_a":None, "processed_b":None, "final_output":None} # Optionalキーも初期化 (解答例より)
print(f"--- ファンアウト・ファンイン テスト (入力: {initial_fan_state['input_data']}) ---")
final_fan_result = graph.invoke(initial_fan_state)
print(f"最終出力: {final_fan_result.get('final_output')}")

<details><summary>解答004</summary>

``````python
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph, END

# 状態定義
class FanOutFanInState(TypedDict):
    input_data: str
    processed_a: Optional[str]
    processed_b: Optional[str]
    final_output: Optional[str]

# ノード定義
def start_node(state: FanOutFanInState):
    print(f"[開始ノード] 入力データ: {state['input_data']}")
    return {}

def process_a_node(state: FanOutFanInState):
    result_a = state["input_data"] + "_ProcessedA"
    print(f"  [処理Aノード] 結果: {result_a}")
    return {"processed_a": result_a}

def process_b_node(state: FanOutFanInState):
    result_b = state["input_data"] + "_ProcessedB"
    print(f"  [処理Bノード] 結果: {result_b}")
    return {"processed_b": result_b}

def aggregate_node(state: FanOutFanInState):
    result = f"集約結果: A='{state.get('processed_a', 'N/A')}', B='{state.get('processed_b', 'N/A')}'"
    print(f"[集約ノード] {result}")
    return {"final_output": result}

# グラフ構築
workflow = StateGraph(FanOutFanInState)
workflow.add_node("start", start_node)
workflow.add_node("process_a", process_a_node)
workflow.add_node("process_b", process_b_node)
workflow.add_node("aggregate", aggregate_node)

workflow.set_entry_point("start")

workflow.add_edge("start", "process_a")
workflow.add_edge("start", "process_b") # ファンアウト

workflow.add_edge(["process_a", "process_b"], "aggregate") # ファンイン

workflow.add_edge("aggregate", END)
graph = workflow.compile()

# 実行
initial_fan_state = {"input_data": "初期データ", "processed_a":None, "processed_b":None, "final_output":None} # Optionalキーも初期化
print(f"--- ファンアウト・ファンイン テスト (入力: {initial_fan_state['input_data']}) ---")
final_fan_result = graph.invoke(initial_fan_state)
print(f"最終出力: {final_fan_result.get('final_output')}")
``````
</details>

<details><summary>解説004</summary>

#### この問題のポイント

*   **ファンアウト:** `start_node` から `process_a_node` と `process_b_node` の両方にエッジを接続することで、`start_node` の処理完了後、これら2つのノードが（理論上）並列に実行されます。
*   **ファンイン:** `add_edge` の第1引数にノード名のリスト `["process_a", "process_b"]` を指定し、第2引数に `aggregate_node` を指定することで、`process_a_node` と `process_b_node` の**両方の処理が完了するのを待ってから** `aggregate_node` が実行されます。
*   これにより、複数の独立した処理を並行して行い、その結果を後で統合する、といったワークフローを効率的に構築できます。

---</details>

### ■ 問題005: Human-in-the-Loop (人間による介在と承認)
*   **学習内容:** `Interrupt` を使用してグラフの実行を一時停止し、人間の確認・承認を得てから処理を再開または分岐させる方法を学びます。

In [ ]:
# 解答欄005 - グラフ構築
____ typing ____ ____, ____, Optional
____ langgraph.graph ____ ____, ____, ____
____ uuid ____ uuid4
____ langgraph.checkpoint.memory ____ ____ # MemorySaverを使う場合 (解答例より)

# 状態定義
____ HumanApprovalState(____):
    task_description: str
    generated_plan: Optional[str]
    human_feedback: Optional[str] # "approve" or "reject" or other comments
    final_result: Optional[str]
    log: list[str]

# ノード定義
____ plan_generation_node(state: HumanApprovalState):
    # ここではダミーの計画生成
    plan = f"タスク「{state['task_description']}」に対する計画案: ステップ1 -> ステップ2 -> ステップ3"
    print(f"[計画生成ノード] 生成された計画: {plan}")
    ____ {"generated_plan": plan, "log": state["log"] + [f"計画生成: {plan}"]}

____ wait_for_human_approval_node(state: HumanApprovalState):
    print(f"
[人間承認待機ノード] 以下の計画について承認またはフィードバックを待っています:")
    print(f"  計画: {state['generated_plan']}")
    # Interruptを発生させて処理を一時停止 (解答例ではcompile時に指定)
    # raise ____() # ここでraiseするか、compile時にinterrupt_beforeを指定
    return {} # 何も状態を更新しない (解答例より)

def process_approved_plan_node(state: HumanApprovalState):
    approved_plan = state["generated_plan"]
    # 承認された計画を実行するダミー処理
    result = f"計画「{approved_plan}」が承認され、正常に実行されました。"
    print(f"[計画実行ノード] {result}")
    return {"final_result": result, "log": state["log"] + [result]}

def process_rejected_plan_node(state: HumanApprovalState):
    rejected_plan = state["generated_plan"]
    feedback = state.get("human_feedback", "(フィードバックなし)")
    result = f"計画「{rejected_plan}」は拒否されました。フィードバック: {feedback}"
    print(f"[計画拒否処理ノード] {result}")
    return {"final_result": result, "log": state["log"] + [result]}

# ルーター関数
def route_after_human_feedback(state: HumanApprovalState):
    feedback = state.get("human_feedback", "").lower()
    if "approve" in feedback:
        print("  -> 人間が承認したため、計画を実行します。")
        return "approved"
    else:
        print("  -> 人間が拒否または他のフィードバックを与えたため、拒否処理を行います。")
        return "rejected"

# グラフ構築
workflow = StateGraph(HumanApprovalState)
workflow.add_node("plan_generator", plan_generation_node)
workflow.add_node("human_approval_step", wait_for_human_approval_node)
workflow.add_node("execute_approved", process_approved_plan_node)
workflow.add_node("handle_rejection", process_rejected_plan_node)

workflow.set_entry_point("plan_generator")
workflow.add_edge("plan_generator", "human_approval_step")

workflow.add_conditional_edges(
    "human_approval_step",
    route_after_human_feedback,
    {
        "approved": "execute_approved",
        "rejected": "handle_rejection" # 解答例より
    }
)
workflow.add_edge("execute_approved", END)
workflow.add_edge("handle_rejection", END)

memory = MemorySaver() # 解答例より
graph = workflow.compile(checkpointer=memory, interrupt_before=["human_approval_step"]) # Interrupt設定 (解答例より)

In [ ]:
# 解答欄005 - グラフ可視化
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# 解答欄005 - グラフ実行
task = "新しいマーケティングキャンペーンを開始する"
initial_approval_state = {
    "task_description": task,
    "log": [],
    "generated_plan": None, # Optionalキーも初期化 (解答例より)
    "human_feedback": None,
    "final_result": None
}

print(f"--- Human-in-the-Loopテスト (タスク: {task}) ---")
thread_id = str(uuid4()) # 各実行にユニークなIDを割り当てる
config = {"configurable": {"thread_id": thread_id}}

print("グラフを初期状態で実行します...")
graph.invoke(initial_approval_state, config=config)

print("
--- 中断後のグラフの状態 --- (Interruptが発生したはず)")
current_graph_state = graph.get_state(config)
if current_graph_state: # 解答例よりNoneチェック追加
    print(f"現在の計画: {current_graph_state.values.get('generated_plan')}")
    print(f"現在のログ: {current_graph_state.values.get('log')}")
else:
    print("中断状態が取得できませんでした。")

# ユーザーからのフィードバックをシミュレート
user_choice = input("計画を承認しますか？ (approve / reject [フィードバック]): ").strip()

feedback_to_inject = {}
if user_choice.lower().startswith("approve"):
    feedback_to_inject = {"human_feedback": "approve"}
elif user_choice.lower().startswith("reject"):
    feedback_to_inject = {"human_feedback": user_choice} # reject [フィードバック] 全体を渡す
else:
    feedback_to_inject = {"human_feedback": f"other: {user_choice}"}

print(f"
注入するフィードバック: {feedback_to_inject}")
final_approval_result = graph.invoke(feedback_to_inject, config=config) # 中断したところから再開

print("
--- 最終結果 ---")
if final_approval_result: # 解答例よりNoneチェック追加
    print(f"最終的な結果メッセージ: {final_approval_result.get('final_result')}")
    print(f"最終ログ: {final_approval_result.get('log')}")
else:
    print("最終状態が取得できませんでした。")

<details><summary>解答005</summary>

``````python
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END, Interrupt
from uuid import uuid4
from langgraph.checkpoint.memory import MemorySaver # MemorySaverを使う場合

# 状態定義
class HumanApprovalState(TypedDict):
    task_description: str
    generated_plan: Optional[str]
    human_feedback: Optional[str]
    final_result: Optional[str]
    log: list[str]

# ノード定義
def plan_generation_node(state: HumanApprovalState):
    plan = f"タスク「{state['task_description']}」に対する計画案: ステップ1 -> ステップ2 -> ステップ3"
    print(f"[計画生成ノード] 生成された計画: {plan}")
    return {"generated_plan": plan, "log": state["log"] + [f"計画生成: {plan}"]}

def wait_for_human_approval_node(state: HumanApprovalState):
    print(f"
[人間承認待機ノード] 以下の計画について承認またはフィードバックを待っています:")
    print(f"  計画: {state['generated_plan']}")
    # このノードが interrupt_before に指定されているため、実行前に Interrupt が発生
    # ここで raise Interrupt() を書く必要はない (compile時に指定するため)
    # もしノードの処理の途中で中断したい場合は raise Interrupt() を使う
    return {} # 何も状態を更新しないが、Interrupt後に再開されることを示す

def process_approved_plan_node(state: HumanApprovalState):
    approved_plan = state["generated_plan"]
    result = f"計画「{approved_plan}」が承認され、正常に実行されました。"
    print(f"[計画実行ノード] {result}")
    return {"final_result": result, "log": state["log"] + [result]}

def process_rejected_plan_node(state: HumanApprovalState):
    rejected_plan = state["generated_plan"]
    feedback = state.get("human_feedback", "(フィードバックなし)")
    result = f"計画「{rejected_plan}」は拒否されました。フィードバック: {feedback}"
    print(f"[計画拒否処理ノード] {result}")
    return {"final_result": result, "log": state["log"] + [result]}

# ルーター関数
def route_after_human_feedback(state: HumanApprovalState):
    feedback = state.get("human_feedback", "").lower()
    if "approve" in feedback:
        print("  -> 人間が承認したため、計画を実行します。")
        return "approved"
    else:
        print("  -> 人間が拒否または他のフィードバックを与えたため、拒否処理を行います。")
        return "rejected"

# グラフ構築
workflow = StateGraph(HumanApprovalState)
workflow.add_node("plan_generator", plan_generation_node)
workflow.add_node("human_approval_step", wait_for_human_approval_node)
workflow.add_node("execute_approved", process_approved_plan_node)
workflow.add_node("handle_rejection", process_rejected_plan_node)

workflow.set_entry_point("plan_generator")
workflow.add_edge("plan_generator", "human_approval_step")

workflow.add_conditional_edges(
    "human_approval_step",
    route_after_human_feedback,
    {
        "approved": "execute_approved",
        "rejected": "handle_rejection"
    }
)
workflow.add_edge("execute_approved", END)
workflow.add_edge("handle_rejection", END)

# MemorySaverインスタンスを作成
memory = MemorySaver()

# interrupt_beforeにノード名を指定し、checkpointerにMemorySaverインスタンスを渡す
graph = workflow.compile(checkpointer=memory, interrupt_before=["human_approval_step"])

# 実行 (インタラクティブ)
task = "新しいマーケティングキャンペーンを開始する"
initial_approval_state = {
    "task_description": task,
    "log": [],
    "generated_plan": None, # Optionalキーも初期化
    "human_feedback": None,
    "final_result": None
}

print(f"--- Human-in-the-Loopテスト (タスク: {task}) ---")
thread_id = str(uuid4()) 
config = {"configurable": {"thread_id": thread_id}}

print("グラフを初期状態で実行します...")
# invokeを呼び出すと、interrupt_beforeで指定したノードの手前でInterruptが発生する
graph.invoke(initial_approval_state, config=config)

print("
--- 中断後のグラフの状態 --- (Interruptが発生しました)")
current_graph_state = graph.get_state(config)
if current_graph_state:
    print(f"現在の計画: {current_graph_state.values.get('generated_plan')}")
    print(f"現在のログ: {current_graph_state.values.get('log')}")
else:
    print("中断状態が取得できませんでした。")

user_choice = input("計画を承認しますか？ (approve / reject [フィードバック]): ").strip()

feedback_to_inject = {}
if user_choice.lower().startswith("approve"):
    feedback_to_inject = {"human_feedback": "approve"}
elif user_choice.lower().startswith("reject"):
    feedback_to_inject = {"human_feedback": user_choice}
else:
    feedback_to_inject = {"human_feedback": f"other: {user_choice}"}

print(f"
注入するフィードバック: {feedback_to_inject}")
# 中断したグラフにフィードバックを与えて再開
# invokeの第一引数はNoneでも良いが、更新する状態を渡すのが一般的
final_approval_result_state = graph.invoke(feedback_to_inject, config=config)

print("
--- 最終結果 ---")
if final_approval_result_state:
    print(f"最終的な結果メッセージ: {final_approval_result_state.get('final_result')}")
    print(f"最終ログ: {final_approval_result_state.get('log')}")
else:
    print("最終状態が取得できませんでした。")
``````
</details>

<details><summary>解説005</summary>

#### この問題のポイント

*   **`Interrupt` と `checkpointer`:** `graph.compile()`時に `interrupt_before=["node_name"]` を指定すると、指定されたノードの実行直前でグラフの実行が一時停止（`Interrupt`）します。この機能を利用するには、`checkpointer`（例: `MemorySaver`）の設定が必須です。`checkpointer`はグラフの状態を保存・復元する役割を担います。
*   **グラフの実行と中断:** `graph.invoke(initial_state, config)` を実行すると、`human_approval_step` ノードの手前で中断します。`config` には `{"configurable": {"thread_id": "unique_id"}}` のようにスレッドIDを指定し、中断と再開を同じ会話（状態）コンテキストで行えるようにします。
*   **状態の取得と更新:** 中断後、`graph.get_state(config)` で現在のグラフの状態を取得できます。ユーザーからのフィードバック（承認/拒否）を新しい状態（`feedback_to_inject`）として用意し、再び `graph.invoke(feedback_to_inject, config)` を呼び出すことで、中断した箇所から処理を再開します。この際、`human_approval_step` ノード自体は中断前に実行されていないため、再開後に実行され、その後 `route_after_human_feedback` ルーターが新しいフィードバックに基づいて処理を分岐させます。
*   **`wait_for_human_approval_node` の役割:** このノードは、`interrupt_before` で指定されているため、実際にはその中身が実行される前に中断が発生します。再開後、このノードが（もし何か処理があれば）実行され、その後エッジとルーターが評価されます。

---</details>

### ■ 問題006: エラーハンドリングとリトライ処理
*   **学習内容:** `try-except`ブロックでエラーを捕捉し、状態に記録します。さらに、そのエラー情報に基づいて、指定回数だけ処理をリトライするループを実装します。

In [ ]:
# 解答欄006 - グラフ構築
____ typing ____ ____, Optional, ____
____ random
____ langgraph.graph ____ ____, ____

# 状態定義
____ RetryState(____):
    data_to_process: str
    processed_result: Optional[str]
    error_message: Optional[str]
    retry_count: int
    max_retries: int
    log: list[str]

# ノード定義
____ potentially_failing_node(state: RetryState):
    print(f"
[処理試行ノード] データ: '{state['data_to_process']}', 試行回数: {state['retry_count'] + 1}")
    # 最初の (max_retries - 1) 回までは高確率で失敗させるための調整 (解答例より)
    should_fail_this_time = random.random() < 0.8 # 80%の確率で失敗を試みる (解答例より)
    is_last_retry_attempt = (state['retry_count'] + 1) >= state['max_retries']

    ____ should_fail_this_time and not is_last_retry_attempt:
        error_msg = "ランダムエラー発生！処理に失敗しました。"
        print(f"  -> エラー: {error_msg}")
        ____ {
            "error_message": error_msg, 
            "retry_count": state["retry_count"] + 1,
            "log": state["log"] + [f"試行 {state['retry_count'] + 1}: エラー - {error_msg}"]
        }
    ____:
        success_reason = "成功" ____ not is_last_retry_attempt ____ "最後のリトライで成功扱い" # 解答例より
        result = f"データ「{state['data_to_process']}」の処理{success_reason} (試行 {state['retry_count'] + 1} 回目)"
        print(f"  -> {success_reason}: {result}")
        ____ {
            "processed_result": result, 
            "error_message": None, # エラー情報をクリア (解答例より)
            "retry_count": state["retry_count"] + 1,
            "log": state["log"] + [f"試行 {state['retry_count'] + 1}: {success_reason} - {result}"]
        }

____ success_node(state: RetryState):
    msg = f"[成功処理ノード] 最終結果: {state['processed_result']}"
    print(msg)
    ____ {"log": state["log"] + [msg]}

def failure_node(state: RetryState):
    msg = f"[失敗処理ノード] 最大リトライ回数 ({state['max_retries']}) に達しました。最終エラー: {state['error_message']}"
    print(msg)
    return {"log": state["log"] + [msg]}

# ルーター関数
def should_retry_or_fail(state: RetryState):
    if state.get("error_message") and state["retry_count"] < state["max_retries"]:
        print("  -> エラー発生、リトライします。")
        return "retry"
    elif state.get("error_message"):
        print("  -> エラー発生、最大リトライ回数超過。失敗処理へ。")
        return "fail"
    else: # エラーなし (成功)
        print("  -> 処理成功。成功処理へ。")
        return "succeed" # 成功時のキー (解答例より)

# グラフ構築
workflow = StateGraph(RetryState)
workflow.add_node("process_data", potentially_failing_node)
workflow.add_node("handle_success", success_node)
workflow.add_node("handle_failure", failure_node)

workflow.set_entry_point("process_data")

workflow.add_conditional_edges(
    "process_data",
    should_retry_or_fail,
    {
        "retry": "process_data",
        "succeed": "handle_success", # 成功時はこのノードへ (解答例より)
        "fail": "handle_failure"
    }
)
workflow.add_edge("handle_success", END)
workflow.add_edge("handle_failure", END)
graph = workflow.compile()

In [ ]:
# 解答欄006 - グラフ可視化
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# 解答欄006 - グラフ実行
data = "重要なデータ"
retries = 3
initial_retry_state = {
    "data_to_process": data,
    "retry_count": 0,
    "max_retries": retries,
    "log": [],
    "processed_result": None, # Optionalキーも初期化
    "error_message": None
}
print(f"--- エラーハンドリングとリトライテスト (データ: {data}, 最大リトライ: {retries}) ---")
final_retry_result = graph.invoke(initial_retry_state, {"recursion_limit": 15})
print("
最終ログ:")
for entry in final_retry_result['log']:
    print(f"  - {entry}")

<details><summary>解答006</summary>

``````python
from typing import TypedDict, Optional, Annotated
import random
from langgraph.graph import StateGraph, END

# 状態定義
class RetryState(TypedDict):
    data_to_process: str
    processed_result: Optional[str]
    error_message: Optional[str]
    retry_count: int
    max_retries: int
    log: list[str]

# ノード定義
def potentially_failing_node(state: RetryState):
    print(f"
[処理試行ノード] データ: '{state['data_to_process']}', 試行回数: {state['retry_count'] + 1}")
    # 最初の (max_retries - 1) 回までは高確率で失敗させるための調整
    # これにより、少なくとも数回のリトライが試みられる可能性を高める
    # 例: max_retries = 3 の場合、retry_count が 0, 1 の時は失敗しやすく、2 (最後のリトライ)では成功しやすくする
    should_fail_this_time = random.random() < 0.8 # 80%の確率で失敗を試みる
    is_last_retry_attempt = (state['retry_count'] + 1) >= state['max_retries']

    if should_fail_this_time and not is_last_retry_attempt:
        error_msg = "ランダムエラー発生！処理に失敗しました。"
        print(f"  -> エラー: {error_msg}")
        return {
            "error_message": error_msg, 
            "retry_count": state["retry_count"] + 1,
            "log": state["log"] + [f"試行 {state['retry_count'] + 1}: エラー - {error_msg}"]
        }
    else:
        # 成功、または最後のリトライでは強制的に成功させる（デモのため）
        success_reason = "成功" if not is_last_retry_attempt else "最後のリトライで成功扱い"
        result = f"データ「{state['data_to_process']}」の処理{success_reason} (試行 {state['retry_count'] + 1} 回目)"
        print(f"  -> {success_reason}: {result}")
        return {
            "processed_result": result, 
            "error_message": None, 
            "retry_count": state["retry_count"] + 1,
            "log": state["log"] + [f"試行 {state['retry_count'] + 1}: {success_reason} - {result}"]
        }

def success_node(state: RetryState):
    msg = f"[成功処理ノード] 最終結果: {state['processed_result']}"
    print(msg)
    return {"log": state["log"] + [msg]}

def failure_node(state: RetryState):
    msg = f"[失敗処理ノード] 最大リトライ回数 ({state['max_retries']}) に達しました。最終エラー: {state['error_message']}"
    print(msg)
    return {"log": state["log"] + [msg]}

# ルーター関数
def should_retry_or_fail(state: RetryState):
    if state.get("error_message") and state["retry_count"] < state["max_retries"]:
        print("  -> エラー発生、リトライします。")
        return "retry"
    elif state.get("error_message"):
        print("  -> エラー発生、最大リトライ回数超過。失敗処理へ。")
        return "fail"
    else: 
        print("  -> 処理成功。成功処理へ。")
        return "succeed"

# グラフ構築
workflow = StateGraph(RetryState)
workflow.add_node("process_data", potentially_failing_node)
workflow.add_node("handle_success", success_node)
workflow.add_node("handle_failure", failure_node)

workflow.set_entry_point("process_data")

workflow.add_conditional_edges(
    "process_data",
    should_retry_or_fail,
    {
        "retry": "process_data",
        "succeed": "handle_success",
        "fail": "handle_failure"
    }
)
workflow.add_edge("handle_success", END)
workflow.add_edge("handle_failure", END)
graph = workflow.compile()

# 実行
data = "重要なデータ"
retries = 3
initial_retry_state = {
    "data_to_process": data,
    "retry_count": 0,
    "max_retries": retries,
    "log": [],
    "processed_result": None,
    "error_message": None
}
print(f"--- エラーハンドリングとリトライテスト (データ: {data}, 最大リトライ: {retries}) ---")
final_retry_result = graph.invoke(initial_retry_state, {"recursion_limit": 2 * retries + 5}) # リトライ回数に応じて調整
print("
最終ログ:")
for entry in final_retry_result['log']:
    print(f"  - {entry}")
``````
</details>

<details><summary>解説006</summary>

#### この問題のポイント

*   **エラーの捕捉と状態への記録:** `potentially_failing_node` 内で、ダミーの処理が失敗すると（ここでは`random.random()`でシミュレート）、`error_message` 状態キーにエラー情報が記録され、`retry_count` がインクリメントされます。成功した場合は `processed_result` が設定され、`error_message` はクリア（または `None` に）されます。
*   **リトライ制御ルーター:** `should_retry_or_fail` ルーターが、`error_message` の有無と `retry_count` が `max_retries` に達しているかどうかに基づいて、次の遷移先を決定します。
    *   エラーがあり、かつリトライ回数が上限未満なら `"retry"` を返し、再び `process_data` ノードを実行します（リトライ）。
    *   エラーがあり、かつリトライ回数が上限に達していたら `"fail"` を返し、`handle_failure` ノードに進みます。
    *   エラーがなければ `"succeed"` を返し、`handle_success` ノードに進みます。
*   **ループによるリトライ:** `"retry"` の場合に `process_data` ノード自身に戻るエッジが、リトライのループを形成します。
*   **`recursion_limit` の調整:** リトライ処理のようにループが発生する可能性があるグラフでは、`graph.invoke` や `graph.stream` の `config` で `recursion_limit` を十分に大きな値に設定する必要があります。デフォルトのままだと、多数回のリトライで再帰深度エラーが発生することがあります。

---</details>

### ■ 問題007: 第2章のまとめ - 反復的な改善と最終承認フローの構築
*   **学習内容:** 第2章で学んだ要素（LLMによる改善ループ、エラーハンドリング、人間による最終承認）を組み合わせ、実践的な多段階プロセスを構築します。

In [ ]:
# 解答欄007 - グラフ構築
____ typing ____ ____, Optional, ____, List
____ random
____ uuid ____ uuid4
____ langgraph.graph ____ ____, ____, ____
____ langgraph.checkpoint.memory ____ ____ # MemorySaverが必要

# 状態定義
____ IterativeApprovalState(____):
    original_request: str
    current_draft: Optional[str]
    llm_critique: Optional[str]
    human_feedback: Optional[str]
    iteration_count: int
    max_iterations: int
    error_details: Optional[str]
    final_document: Optional[str]
    log: List[str]

# ノード定義
____ initial_draft_node(state: IterativeApprovalState):
    print(f"
[初期ドラフト生成] リクエスト: {state['original_request']}")
    current_iter = state.get("iteration_count", 0) # 解答例より
    draft = f"「{state['original_request']}」に関するドラフトです。バージョン{current_iter + 1}。"
    ____ state.get("human_feedback") and "revise" in state["human_feedback"].lower(): # 解答例より
        draft += f" (フィードバック「{state['human_feedback']}」を反映しました)"
    ____ state.get("llm_critique") and "完璧" not in state["llm_critique"].lower(): # 解答例より
        draft += f" (LLMレビュー「{state['llm_critique'][:30]}...」を反映しました)"

    if random.random() < 0.1 and current_iter == 0: # 低確率で初回のみエラー (解答例より)
        err = "初期ドラフト生成中に予期せぬAPIエラーが発生しました。"
        print(f"  -> エラー: {err}")
        return {"error_details": err, "iteration_count": current_iter + 1, "log": state["log"] + [f"ドラフト(v{current_iter+1})エラー: {err}"]}
    print(f"  -> 生成ドラフト: {draft}")
    return {"current_draft": draft, "iteration_count": current_iter + 1, "log": state["log"] + [f"ドラフト生成(v{current_iter+1}): {draft}"]
            , "error_details": None, "llm_critique": None, "human_feedback": None} # 解答例よりリセット

def llm_review_node(state: IterativeApprovalState):
    if state.get("error_details"): return {} # 前のノードでエラーなら何もしない
    draft = state["current_draft"]
    print(f"
[LLMレビュー] ドラフト: {draft}")
    # ダミーLLM評価
    if state["iteration_count"] < state["max_iterations"] / 2: # 初期は厳しめにレビュー (解答例より)
        crit = f"改善点あり: もっと詳細な情報が必要です (反復 {state['iteration_count']})。"
    elif state["iteration_count"] < state["max_iterations"]: # 解答例より
        crit = f"ほぼ良いですが、結論部分を強調してください (反復 {state['iteration_count']})。"
    else:
        crit = "完璧です。これ以上改善の必要はありません。"
    print(f"  -> LLM評価: {crit}")
    return {"llm_critique": crit, "log": state["log"] + [f"LLMレビュー: {crit}"]}

def human_approval_gateway_node(state: IterativeApprovalState):
    if state.get("error_details"): return {} # エラー時は承認ステップをスキップ
    print(f"
[人間による承認ゲートウェイ] ドラフト: {state['current_draft']}")
    print(f"  LLMレビュー: {state.get('llm_critique', 'N/A')}")
    return {}

def finalize_document_node(state: IterativeApprovalState):
    final_doc = state["current_draft"]
    feedback = state.get("human_feedback", "(フィードバックなし)")
    msg = f"[最終化] ドキュメント「{final_doc}」が人間によって承認されました。フィードバック: {feedback}"
    print(msg)
    return {"final_document": final_doc, "log": state["log"] + [msg]}

def revision_or_rejection_node(state: IterativeApprovalState):
    feedback = state.get("human_feedback", "")
    action = "改訂" if "revise" in feedback.lower() else "最終拒否" # 解答例より
    msg = f"[{action}処理] 人間のフィードバック: 「{feedback}」。"
    print(msg)
    return {"log": state["log"] + [msg]} # current_draftの更新はinitial_draft_nodeで行う (解答例より)

def error_processing_node(state: IterativeApprovalState):
    err_msg = f"[エラー処理] エラー発生: {state['error_details']}"
    print(err_msg)
    return {"log": state["log"] + [err_msg], "final_document": "エラーのため処理中断"} # 解答例より

# ルーター関数
def route_after_llm_review(state: IterativeApprovalState):
    if state.get("error_details"): return "error_handler"
    critique = state.get("llm_critique", "")
    if "完璧" in critique.lower() or state["iteration_count"] >= state["max_iterations"]:
        print("  -> LLMレビューOKまたは最大反復。人間承認へ。")
        return "human_approval"
    else:
        print("  -> LLMレビューで改善点あり。再ドラフトへ。")
        return "regenerate_draft" # 再ドラフトのキー (解答例より)

def route_after_human_feedback(state: IterativeApprovalState):
    feedback = state.get("human_feedback", "").lower()
    if "approve" in feedback:
        print("  -> 人間が承認。最終化へ。")
        return "approved_by_human"
    elif "revise" in feedback and state["iteration_count"] < state["max_iterations"]: # 解答例より
        print("  -> 人間が改訂を要求。revision_handler経由で再ドラフトへ。")
        return "needs_revision"
    else: # reject or revise but max_iterations reached (解答例より)
        print("  -> 人間が拒否、または改訂上限。revision_handlerで終了。")
        return "rejected_by_human"

# グラフ構築
workflow = StateGraph(IterativeApprovalState)
workflow.add_node("draft", initial_draft_node)
workflow.add_node("llm_review", llm_review_node)
workflow.add_node("human_approval", human_approval_gateway_node)
workflow.add_node("finalize", finalize_document_node)
workflow.add_node("revision_handler", revision_or_rejection_node) 
workflow.add_node("error_handler", error_processing_node)

workflow.set_entry_point("draft")
workflow.add_edge("draft", "llm_review")

workflow.add_conditional_edges(
    "llm_review", route_after_llm_review,
    {
        "regenerate_draft": "draft", 
        "human_approval": "human_approval",
        "error_handler": "error_handler"
    }
)
workflow.add_conditional_edges(
    "human_approval", route_after_human_feedback,
    {
        "approved_by_human": "finalize",
        "needs_revision": "revision_handler", # 解答例より revision_handler を経由
        "rejected_by_human": "revision_handler"
    }
)
workflow.add_edge("revision_handler", "draft") # 改訂指示の場合、revision_handlerの後に再度draftへ (解答例より)
workflow.add_edge("finalize", END)
workflow.add_edge("error_handler", END)   # エラー処理後終了

chapter2_summary_memory = MemorySaver() # 解答例より
graph = workflow.compile(checkpointer=chapter2_summary_memory, interrupt_before=["human_approval"])

In [ ]:
# 解答欄007 - グラフ可視化
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# 解答欄007 - グラフ実行
request = "LangGraphを使った新しいAIエージェントの提案書"
iterations = 2 # LLMによる自動反復の上限回数（人間による改訂要求はこの回数に含まれない）(解答例より)
ch2_thread_id = str(uuid4())
ch2_config = {"configurable": {"thread_id": ch2_thread_id}}
initial_ch2_state = {
    "original_request": request,
    "iteration_count": 0,
    "max_iterations": iterations,
    "log": [],
    "current_draft":None, "llm_critique":None, "human_feedback":None, "error_details":None, "final_document":None
}
print(f"--- 第2章まとめテスト (リクエスト: {request}, LLM最大反復: {iterations}) ---")

current_graph_run = graph.invoke(initial_ch2_state, config=ch2_config) # 解答例の実行ロジックを適用

if graph.get_state(ch2_config).next: # Check if graph is interrupted (解答例より)
    current_ch2_state_values = graph.get_state(ch2_config).values
    print(f"
--- 人間による確認 --- (現在のドラフト: {current_ch2_state_values.get('current_draft')})")
    human_input = input("ドラフトを承認しますか？ (approve / revise [指示] / reject): ").strip()
    
    current_graph_run = graph.invoke({"human_feedback": human_input}, config=ch2_config)
    while "revise" in human_input.lower() and graph.get_state(ch2_config).next and current_graph_run.get("iteration_count", 0) < current_graph_run.get("max_iterations", iterations) +1: # 解答例のループ条件
        print("人間による改訂要求後、再度LLMレビューと承認ゲートウェイに到達しました。")
        current_ch2_state_values = graph.get_state(ch2_config).values
        print(f"  改訂後ドラフト: {current_ch2_state_values.get('current_draft')}")
        print(f"  LLMレビュー: {current_ch2_state_values.get('llm_critique')}")
        human_input = input("改訂版ドラフトを承認しますか？ (approve / revise [指示] / reject): ").strip()
        current_graph_run = graph.invoke({"human_feedback": human_input}, config=ch2_config)
        if "approve" in human_input.lower() or "reject" in human_input.lower():
            break

print("
--- 最終結果 --- ")
if current_graph_run.get("final_document"):
    print(f"最終ドキュメント: {current_graph_run['final_document']}")
elif current_graph_run.get("error_details"):
    print(f"エラー終了: {current_graph_run['error_details']}")
else:
    print(f"処理完了。最終ドラフト: {current_graph_run.get('current_draft')}")
    if current_graph_run.get("human_feedback"):
        print(f"  最後の人間フィードバック: {current_graph_run.get('human_feedback')}")

print("
全ログ:")
for entry in current_graph_run.get('log', []):
    print(f"  - {entry}")

<details><summary>解答007</summary>

``````python
from typing import TypedDict, Optional, Annotated, List
import random
from uuid import uuid4
from langgraph.graph import StateGraph, END, Interrupt
from langgraph.checkpoint.memory import MemorySaver

# 状態定義
class IterativeApprovalState(TypedDict):
    original_request: str
    current_draft: Optional[str]
    llm_critique: Optional[str]
    human_feedback: Optional[str]
    iteration_count: int
    max_iterations: int
    error_details: Optional[str]
    final_document: Optional[str]
    log: List[str]

# ノード定義
def initial_draft_node(state: IterativeApprovalState):
    print(f"
[初期ドラフト生成] リクエスト: {state['original_request']}")
    current_iter = state.get("iteration_count", 0)
    draft = f"「{state['original_request']}」に関するドラフトです。バージョン{current_iter + 1}。"
    # 前回のフィードバックがあればそれを反映する（ダミー処理）
    if state.get("human_feedback") and "revise" in state["human_feedback"].lower():
        draft += f" (フィードバック「{state['human_feedback']}」を反映しました)"
    elif state.get("llm_critique") and "完璧" not in state["llm_critique"].lower():
        draft += f" (LLMレビュー「{state['llm_critique'][:30]}...」を反映しました)"

    if random.random() < 0.1 and current_iter == 0: # 低確率で初回のみエラー
        err = "初期ドラフト生成中に予期せぬAPIエラーが発生しました。"
        print(f"  -> エラー: {err}")
        return {"error_details": err, "iteration_count": current_iter + 1, "log": state["log"] + [f"ドラフト(v{current_iter+1})エラー: {err}"]}
    print(f"  -> 生成ドラフト: {draft}")
    return {"current_draft": draft, "iteration_count": current_iter + 1, "log": state["log"] + [f"ドラフト生成(v{current_iter+1}): {draft}"]
            , "error_details": None, "llm_critique": None, "human_feedback": None}

def llm_review_node(state: IterativeApprovalState):
    if state.get("error_details"): return {} 
    draft = state["current_draft"]
    print(f"
[LLMレビュー] ドラフト: {draft}")
    if state["iteration_count"] < state["max_iterations"] / 2: # 初期は厳しめにレビュー
        crit = f"改善点あり: もっと詳細な情報が必要です (反復 {state['iteration_count']})。"
    elif state["iteration_count"] < state["max_iterations"]:
        crit = f"ほぼ良いですが、結論部分を強調してください (反復 {state['iteration_count']})。"
    else:
        crit = "完璧です。これ以上改善の必要はありません。"
    print(f"  -> LLM評価: {crit}")
    return {"llm_critique": crit, "log": state["log"] + [f"LLMレビュー: {crit}"]}

def human_approval_gateway_node(state: IterativeApprovalState):
    if state.get("error_details"): return {} 
    print(f"
[人間による承認ゲートウェイ] ドラフト: {state['current_draft']}")
    print(f"  LLMレビュー: {state.get('llm_critique', 'N/A')}")
    return {}

def finalize_document_node(state: IterativeApprovalState):
    final_doc = state["current_draft"]
    feedback = state.get("human_feedback", "(フィードバックなし)")
    msg = f"[最終化] ドキュメント「{final_doc}」が人間によって承認されました。フィードバック: {feedback}"
    print(msg)
    return {"final_document": final_doc, "log": state["log"] + [msg]}

def revision_or_rejection_node(state: IterativeApprovalState):
    feedback = state.get("human_feedback", "")
    action = "改訂" if "revise" in feedback.lower() else "最終拒否"
    msg = f"[{action}処理] 人間のフィードバック: 「{feedback}」。"
    print(msg)
    # 改訂の場合、iteration_countは次のループで増えるのでここでは何もしない
    # current_draftは次のinitial_draft_nodeでフィードバックを元に更新される想定
    return {"log": state["log"] + [msg]}

def error_processing_node(state: IterativeApprovalState):
    err_msg = f"[エラー処理] エラー発生: {state['error_details']}"
    print(err_msg)
    return {"log": state["log"] + [err_msg], "final_document": "エラーのため処理中断"}

# ルーター関数
def route_after_llm_review(state: IterativeApprovalState):
    if state.get("error_details"): return "error_handler"
    critique = state.get("llm_critique", "")
    if "完璧" in critique.lower() or state["iteration_count"] >= state["max_iterations"]:
        print("  -> LLMレビューOKまたは最大反復。人間承認へ。")
        return "human_approval"
    else:
        print("  -> LLMレビューで改善点あり。再ドラフトへ。")
        return "regenerate_draft"

def route_after_human_feedback(state: IterativeApprovalState):
    feedback = state.get("human_feedback", "").lower()
    if "approve" in feedback:
        print("  -> 人間が承認。最終化へ。")
        return "approved_by_human"
    elif "revise" in feedback and state["iteration_count"] < state["max_iterations"]:
        print("  -> 人間が改訂を要求。revision_handler経由で再ドラフトへ。")
        return "needs_revision"
    else: # reject or revise but max_iterations reached
        print("  -> 人間が拒否、または改訂上限。revision_handlerで終了。")
        return "rejected_by_human"

# グラフ構築
workflow = StateGraph(IterativeApprovalState)
workflow.add_node("draft", initial_draft_node)
workflow.add_node("llm_review", llm_review_node)
workflow.add_node("human_approval", human_approval_gateway_node)
workflow.add_node("finalize", finalize_document_node)
workflow.add_node("revision_handler", revision_or_rejection_node) 
workflow.add_node("error_handler", error_processing_node)

workflow.set_entry_point("draft")
workflow.add_edge("draft", "llm_review")

workflow.add_conditional_edges(
    "llm_review", route_after_llm_review,
    {
        "regenerate_draft": "draft", 
        "human_approval": "human_approval",
        "error_handler": "error_handler"
    }
)
workflow.add_conditional_edges(
    "human_approval", route_after_human_feedback,
    {
        "approved_by_human": "finalize",
        "needs_revision": "revision_handler", 
        "rejected_by_human": "revision_handler"
    }
)
workflow.add_edge("revision_handler", "draft") # 改訂指示の場合、revision_handlerの後に再度draftへ
workflow.add_edge("finalize", END)
workflow.add_edge("error_handler", END)

chapter2_summary_memory = MemorySaver()
graph = workflow.compile(checkpointer=chapter2_summary_memory, interrupt_before=["human_approval"])

# 実行
request = "LangGraphの高度な機能に関する技術ブログ記事の草案"
iterations = 2 # LLMによる自動反復の上限回数（人間による改訂要求はこの回数に含まれない）
ch2_thread_id = str(uuid4())
ch2_config = {"configurable": {"thread_id": ch2_thread_id}}
initial_ch2_state = {
    "original_request": request,
    "iteration_count": 0,
    "max_iterations": iterations,
    "log": [],
    "current_draft":None, "llm_critique":None, "human_feedback":None, "error_details":None, "final_document":None
}
print(f"--- 第2章まとめテスト (リクエスト: {request}, LLM最大反復: {iterations}) ---")

current_graph_run = graph.invoke(initial_ch2_state, config=ch2_config)

if graph.get_state(ch2_config).next: # Check if graph is interrupted
    current_ch2_state_values = graph.get_state(ch2_config).values
    print(f"
--- 人間による確認 --- (現在のドラフト: {current_ch2_state_values.get('current_draft')})")
    human_input = input("ドラフトを承認しますか？ (approve / revise [指示] / reject): ").strip()
    
    # 人間のフィードバックを注入して再開
    current_graph_run = graph.invoke({"human_feedback": human_input}, config=ch2_config)
    # 人間がreviseを選び、まだiteration_count < max_iterations の場合、再度中断する可能性がある
    while "revise" in human_input.lower() and graph.get_state(ch2_config).next and current_graph_run.get("iteration_count", 0) < current_graph_run.get("max_iterations", iterations) +1: # +1 for human revise loop
        print("人間による改訂要求後、再度LLMレビューと承認ゲートウェイに到達しました。")
        current_ch2_state_values = graph.get_state(ch2_config).values
        print(f"  改訂後ドラフト: {current_ch2_state_values.get('current_draft')}")
        print(f"  LLMレビュー: {current_ch2_state_values.get('llm_critique')}")
        human_input = input("改訂版ドラフトを承認しますか？ (approve / revise [指示] / reject): ").strip()
        current_graph_run = graph.invoke({"human_feedback": human_input}, config=ch2_config)
        if "approve" in human_input.lower() or "reject" in human_input.lower():
            break

print("
--- 最終結果 --- ")
if current_graph_run.get("final_document"):
    print(f"最終ドキュメント: {current_graph_run['final_document']}")
elif current_graph_run.get("error_details"):
    print(f"エラー終了: {current_graph_run['error_details']}")
else:
    print(f"処理完了。最終ドラフト: {current_graph_run.get('current_draft')}")
    if current_graph_run.get("human_feedback"):
        print(f"  最後の人間フィードバック: {current_graph_run.get('human_feedback')}")

print("
全ログ:")
for entry in current_graph_run.get('log', []):
    print(f"  - {entry}")
``````
</details>

<details><summary>解説007</summary>

#### この問題のポイント

*   **複合的なフロー:** このグラフは、初期ドラフト生成、LLMによるレビュー、人間による承認という複数のステージを経ます。
*   **LLM改善ループ:** `draft` -> `llm_review` -> (改善点あれば) `draft` というループが形成され、LLMが自己（または他のLLM）の生成物を反復的に改善しようとします。`max_iterations` でループ回数を制限しています。
*   **エラーハンドリング:** `initial_draft_node` で発生しうるエラーを捕捉し、`error_processing_node` に処理を分岐させています。
*   **人間による介入:** `human_approval_gateway_node` の手前で `Interrupt` が発生し、人間の入力を待ちます。入力 (`human_feedback`) に応じて、承認なら `finalize_document_node` へ、改訂要求なら `revision_or_rejection_node` を経由して再度 `draft` へ（改訂ループ）、拒否なら `revision_or_rejection_node` を経由して終了します。
*   **状態遷移の複雑さ:** 複数の条件分岐とループが組み合わさることで、状態遷移のパスが複雑になります。このような場合にグラフの可視化が非常に役立ちます。
*   **`MemorySaver` と `config`:** `Interrupt` を使用する際は `checkpointer` が必須です。ここでは `MemorySaver` を使用し、`invoke` 時に `config` でスレッドIDを指定することで、中断と再開の状態を正しく管理しています。

この問題は第2章で学んだ多くの要素を組み合わせた実践的な例であり、LangGraphの柔軟性と表現力を示しています。

---</details>